# Transformer-based baseline (BERT)

A transformer-based sentiment classification model is evaluated using a pre-trained BERT architecture. This notebook establishes a deep learning baseline to compare against classical machine learning approaches based on TF-IDF features.

## Load dataset and create train–test split

The same cleaned and balanced review dataset used for the classical machine learning baselines is loaded. The train–test split is reproduced to ensure a fair comparison between models.

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the cleaned dataset
df = pd.read_csv("../data/balanced_reviews.csv")

In [21]:
# Double check column names
df.columns

Index(['Text', 'Sentiment'], dtype='object')

In [22]:
# Extract text and labels
X = df["Text"]
y = df["Sentiment"]

# Create a reproducible train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

## Tokenisation using a pre-trained BERT tokenizer
Text data is converted into tokens that the BERT model can understand. This step prepares the text for input into the BERT model by converting words into numerical representations.

In [23]:
import os

# Set a local Hugging Face cache directory inside the project
os.environ["HF_HOME"] = os.path.join(os.getcwd(), ".hf_cache")
os.environ["TRANSFORMERS_CACHE"] = os.path.join(os.getcwd(), ".hf_cache")

# Ensure the cache directory exists
os.makedirs(os.environ["HF_HOME"], exist_ok=True)

os.environ["HF_HOME"]

'/Users/tommorton/Library/CloudStorage/OneDrive-Personal/Masters/Comp Sci/Modules/Masters Project/AmazonSentinmentAnalysis/notebooks/.hf_cache'

In [24]:
from transformers import BertTokenizer
import torch

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(
    "bert-base-uncased",
    cache_dir=os.path.join(os.getcwd(), ".hf_cache")
)

# Tokenise the training and test sets
train_encodings = tokenizer(
    X_train.tolist(), 
    truncation=True, 
    padding=True, 
    max_length=128
)
test_encodings = tokenizer(
    X_test.tolist(),
    truncation=True, 
    padding=True, 
    max_length=128
)